In [ ]:
import tkinter as tk
import threading


In [3]:
import time
import random
import redis

# Configuración de Redis
r = redis.Redis(
    host='redis-18290.c57.us-east-1-4.ec2.redns.redis-cloud.com',
    port=18290,
    password='NcAH7LtarBZflWxhHIi73zmS4mMIbrVv')

# Simulación de los jugadores
jugadores = [
    {"id": 1, "nombre": "Carlos_5485", "puntos": 1200},
    {"id": 2, "nombre": "lazz_111", "puntos": 1500},
    {"id": 3, "nombre": "F555", "puntos": 1300},
    {"id": 4, "nombre": "Michael_S", "puntos": 1800}
]

# Actualiza los puntajes de los jugadores
def actualizar_puntos(jugador_nombre):
    puntos_actuales = r.zscore("ranking_jugadores", jugador_nombre)
    if puntos_actuales is None:
        puntos_actuales = 0    

    nuevos_puntos = random.randint(100, 6000)
    puntos_finales = max(puntos_actuales + nuevos_puntos, 0)
    timestamp = time.time()

    # Actualizar puntajes en Redis
    r.zadd("ranking_jugadores", {jugador_nombre: puntos_finales})
    r.hset(f"{jugador_nombre}_last_update", "timestamp", timestamp)
    
    # Guardar en el historial
    historial_entrada = f"{time.ctime(timestamp)}: {puntos_finales}"
    r.lpush(f"{jugador_nombre}_historial", historial_entrada)

# Función para mostrar el top 3 jugadores en la interfaz
def mostrar_top3_ranking():
    ranking = r.zrevrange("ranking_jugadores", 0, 2, withscores=True)  # Top 3 jugadores
    return [(nombre.decode('utf-8'), puntos) for nombre, puntos in ranking]

# Hilo para actualizar puntajes en segundo plano
def actualizar_puntajes_periodicamente():
    while True:
        jugador_nombre = random.choice([jugador['nombre'] for jugador in jugadores])
        actualizar_puntos(jugador_nombre)
        time.sleep(10)

# Hilo para actualizar la interfaz gráfica
def actualizar_interfaz():
    while True:
        top3 = mostrar_top3_ranking()
        for i, (nombre, puntos) in enumerate(top3):
            etiquetas[i].config(text=f"{i+1}. {nombre}: {puntos} puntos")
        time.sleep(1)

# Configuración de la interfaz con Tkinter
def crear_interfaz():
    ventana = tk.Tk()
    ventana.title("Ranking de Jugadores")

    # Etiquetas para mostrar el top 3
    global etiquetas
    etiquetas = [tk.Label(ventana, text="Cargando...", font=("Helvetica", 14)) for _ in range(3)]
    for etiqueta in etiquetas:
        etiqueta.pack()

    # Hilos
    hilo_actualizar_puntajes = threading.Thread(target=actualizar_puntajes_periodicamente, daemon=True)
    hilo_actualizar_puntajes.start()

    hilo_interfaz = threading.Thread(target=actualizar_interfaz, daemon=True)
    hilo_interfaz.start()

    # Iniciar la interfaz gráfica
    ventana.mainloop()

# Ejecutar la interfaz
crear_interfaz()


KeyboardInterrupt: 